## Model fitting

In [26]:
# Importing necessary packages
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
import pandas as pd
import joblib


### Data preprocessing

In [31]:

# Load datasets
sales_data = pd.read_csv('sales.csv')
item_description = pd.read_csv('product.csv')
coffee_prices = pd.read_csv('coffee_price.csv')
tea_prices = pd.read_csv('tea_price.csv')

# Preprocessing steps (filtering, merging, encoding) will go here

# Further steps including feature engineering, model training, and evaluation
# will follow after the preprocessing is completed
sales_data

,transaction_id,transaction_date,transaction_time,sales_outlet_id,staff_id,customer_id,instore_yn,order,line_item_id,product_id,quantity,line_item_amount,unit_price,promo_item_yn
0,7,2019-04-01,12:04:43,3,12,558,N,1,1,52,1,2.50,2.50,N
1,11,2019-04-01,15:54:39,3,17,781,N,1,1,27,2,7.00,3.50,N
2,19,2019-04-01,14:34:59,3,17,788,Y,1,1,46,2,5.00,2.50,N
3,32,2019-04-01,16:06:04,3,12,683,N,1,1,23,2,5.00,2.50,N
4,33,2019-04-01,19:18:37,3,17,99,Y,1,1,34,1,2.45,2.45,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49889,753,2019-04-29,16:51:58,8,42,0,N,1,1,30,1,3.00,3.00,N
49890,756,2019-04-29,16:51:14,8,42,8412,Y,1,1,25,2,4.40,2.20,N
49891,759,2019-04-29,11:17:36,8,15,0,Y,1,1,31,1,2.20,2.20,N
49892,763,2019-04-29,15:45:52,8,45,8030,N,1,1,44,2,5.00,2.50,N


In [32]:

coffee_prices

,DATE,PCOFFOTMUSDM
0,1990-01-01,75.829987
1,1990-02-01,84.010010
2,1990-03-01,93.959991
3,1990-04-01,93.730011
4,1990-05-01,92.019989
...,...,...
401,2023-06-01,207.385000
402,2023-07-01,193.494286
403,2023-08-01,186.346957
404,2023-09-01,183.589048


In [30]:

tea_prices


,DATE,PTEAUSDM
0,1990-01-01,252.929993
1,1990-02-01,211.289993
2,1990-03-01,192.600006
3,1990-04-01,194.809998
4,1990-05-01,187.020004
...,...,...
401,2023-06-01,264.999257
402,2023-07-01,273.241209
403,2023-08-01,307.449354
404,2023-09-01,326.762683


In [20]:
# Sum the line_item_amount for each product for each day
grouped_sales = sales_data.groupby(['transaction_date', 'product_id'])['line_item_amount'].sum().reset_index()

# Pivot the table to have product_ids as columns
pivot_sales = grouped_sales.pivot_table(index='transaction_date', columns='product_id', values='line_item_amount', fill_value=0)

# Reset index to make it a flat DataFrame
pivot_sales.reset_index(inplace=True)

# Rename columns for readability
pivot_sales.columns = ['trans_date'] + ['productid_' + str(i) for i in pivot_sales.columns[1:]]

# Add a total amount column
pivot_sales['total_amount_of_the_day'] = pivot_sales.iloc[:, 1:].sum(axis=1)

# Display the result
pivot_sales

,trans_date,productid_1,productid_2,productid_3,productid_4,productid_5,productid_6,productid_7,productid_8,productid_9,...,productid_76,productid_77,productid_78,productid_79,productid_81,productid_82,productid_83,productid_84,productid_87,total_amount_of_the_day
0,2019-04-01,0,0,0.00,0.00,0,0,0.00,0,0.0,...,73.5,69,99.00,63.75,0,0,0,0.0,78.0,7462.00
1,2019-04-02,0,0,0.00,0.00,0,0,0.00,0,0.0,...,59.5,66,81.00,105.00,0,0,0,0.0,126.0,7269.10
2,2019-04-03,0,0,0.00,0.00,0,0,0.00,0,0.0,...,59.5,54,85.50,105.00,0,0,0,0.0,102.0,7461.25
3,2019-04-04,0,0,0.00,0.00,0,0,0.00,0,0.0,...,77.0,42,63.00,67.50,0,0,0,0.0,117.0,7086.45
4,2019-04-05,0,0,0.00,0.00,0,0,0.00,0,0.0,...,77.0,63,72.00,75.00,0,0,0,0.0,96.0,7236.25
5,2019-04-06,0,0,0.00,0.00,0,0,0.00,0,0.0,...,35.0,63,76.50,63.75,0,0,0,0.0,93.0,6513.75
6,2019-04-07,18,36,0.00,61.35,0,42,19.75,45,284.0,...,84.0,57,103.50,75.00,0,0,0,19.2,0.0,7693.73
7,2019-04-08,198,36,73.75,20.45,75,105,79.00,135,45.0,...,122.5,66,189.00,116.25,28,12,70,11.2,30.0,8837.54
8,2019-04-09,90,54,44.25,61.35,45,63,39.50,90,45.0,...,81.4,48,122.65,84.39,84,144,140,0.0,42.0,8355.48
9,2019-04-10,90,72,29.50,20.45,45,21,79.00,180,67.5,...,91.0,66,117.00,78.75,56,72,42,39.2,66.0,8210.48


### Model training

In [36]:


# Load the data
data = pivot_sales

# Define the number of shift days for creating lag features
shift_days = 14

# Prepare the dataset for modeling
def prepare_dataset(data, target_column):
    for i in range(1, shift_days + 1):
        data[f'{target_column}_shifted_{i}'] = data[target_column].shift(i)
    data.dropna(inplace=True)
    return data

# Train and evaluate SVR models for each product ID
r2_scores_train = {}
r2_scores_test = {}

for product_id in range(1, 79):
    product_column = f'productid_{product_id}'

    if product_column in data.columns:
        # Prepare the dataset
        product_data = prepare_dataset(data.copy(), product_column)

        # Split the data
        X = product_data[[f'{product_column}_shifted_{i}' for i in range(1, shift_days + 1)]]
        y = product_data[product_column]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Train the model
        svr_model = SVR()
        svr_model.fit(X_train, y_train)

        # Predict and calculate R^2 score
        y_train_pred = svr_model.predict(X_train)
        r2_train = r2_score(y_train, y_train_pred)

        y_test_pred = svr_model.predict(X_test)
        r2_test = r2_score(y_test, y_test_pred)

        # Store the R^2 scores
        r2_scores_train[product_id] = r2_train
        r2_scores_test[product_id] = r2_test

        # Save the model
        joblib.dump(svr_model, f'svr_model_productid_{product_id}.joblib')

# Print R^2 scores for train and test data
print("Training Data R^2 Scores:", r2_scores_train)
print("Test Data R^2 Scores:", r2_scores_test)




Training Data R^2 Scores: {1: 0.06963843214055143, 2: 0.04151262674779088, 3: 0.07173199223457749, 4: 0.033086750497509754, 5: -0.26151189092496674, 6: 0.02308240395222716, 7: -0.04313978560142151, 8: -0.06889019069162639, 9: 0.0037650362507578583, 10: 0.05066019540001021, 11: -0.08799740256821664, 12: 0.025331868023616755, 13: 0.08422466013349994, 14: 0.07849674185170818, 15: 0.06410805211618698, 16: 0.10953401536659413, 17: 0.08564103595569883, 18: 0.06418904237536804, 19: 0.14713120704897098, 20: 0.15323304600766574, 21: 0.05370957020909528, 22: 0.09073656927248885, 23: 0.02419281728227174, 24: -0.016293849011776107, 25: -0.1317889578712912, 26: 0.04836012218123953, 27: 0.030039732344083414, 28: -0.23338961622111887, 29: -0.0003577333416384665, 30: -0.18849466505454315, 31: 0.034400078390681155, 32: -0.006550495227611197, 33: 0.013478648508276625, 34: -0.0048404776227257695, 35: 0.037964990280550226, 36: 0.023106296654815894, 37: 0.03636301159224542, 38: 0.015083036240851722, 39: 0.

In [37]:


# Define the number of shift days for creating lag features
shift_days = 14

# Prepare the dataset for modeling
def prepare_dataset(data, target_column):
    for i in range(1, shift_days + 1):
        data[f'{target_column}_shifted_{i}'] = data[target_column].shift(i)
    data.dropna(inplace=True)
    return data

# Prepare the dataset for the total sales
total_sales_data = prepare_dataset(data.copy(), 'total_amount_of_the_day')

# Split the data
X = total_sales_data[[f'total_amount_of_the_day_shifted_{i}' for i in range(1, shift_days + 1)]]
y = total_sales_data['total_amount_of_the_day']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Train the model
svr_model_total_sales = SVR()
svr_model_total_sales.fit(X_train, y_train)

# Predict and calculate R^2 score
y_train_pred = svr_model_total_sales.predict(X_train)
r2_train = r2_score(y_train, y_train_pred)

y_test_pred = svr_model_total_sales.predict(X_test)
r2_test = r2_score(y_test, y_test_pred)

# Save the model
joblib.dump(svr_model_total_sales, 'svr_model_total_sales.joblib')

# Print R^2 scores for train and test data
r2_train, r2_test



(-0.0032771265292046436, -1.363246486126339)

In [35]:
# Prepare the data and train a linear regression model
def train_linear_regression(data, column_name):
    # Convert 'DATE' to datetime and extract the year and month as separate columns
    data['DATE'] = pd.to_datetime(data['DATE'])
    data['Year'] = data['DATE'].dt.year
    data['Month'] = data['DATE'].dt.month

    # Prepare the features (X) and target (y)
    X = data[['Year', 'Month']]
    y = data[column_name]

    # Create and train the linear regression model
    model = LinearRegression()
    model.fit(X, y)

    return model

# Load the coffee and tea price data
coffee_price_data = pd.read_csv('coffee_price.csv')
tea_price_data = pd.read_csv('tea_price.csv')

# Train the models for coffee and tea
coffee_model = train_linear_regression(coffee_price_data, 'PCOFFOTMUSDM')
tea_model = train_linear_regression(tea_price_data, 'PTEAUSDM')

# Save the models
joblib.dump(coffee_model, 'coffee_model.joblib')
joblib.dump(tea_model, 'tea_model.joblib')

# Prepare the data for May 2019 prediction
predict_date = pd.DataFrame({'Year': [2019], 'Month': [5]})

# Make predictions for May 2019
predicted_coffee_price = coffee_model.predict(predict_date)
predicted_tea_price = tea_model.predict(predict_date)

print("Predicted Coffee Price for May 2019:", predicted_coffee_price[0])
print("Predicted Tea Price for May 2019:", predicted_tea_price[0])


Predicted Coffee Price for May 2019: 173.57628582026246
Predicted Tea Price for May 2019: 323.19653232269775
